# 9. Convolutional Neural Network with PyTorch

## 1. About Convolutuional Neural Network

### 1.1 Transition From Feedforward Neural Network

#### 1 Hidden Layer Feedforward Neural Network

<img src='./images/09-01.png'>

#### Basic Convolutioanl Neural Network

- Additional **convolution** and **pooling** layers **before feedforward neural network**
- Layer with a **linear function & non-linearity: Fully connected layer**

<img src='./images/09-02.png'>

### 1.2 One Convolutioanl Layer: High Level View

- Input Depth = 1 <==> Single Color Image

<img src = "./images/09-03.png">

<img src = "./images/09-04.png">

<img src = "./images/09-05.png">

<img src = "./images/09-06.png">

<img src = "./images/09-07.png">

<img src = "./images/09-03.png">

- Input Depth = 3 <==> RGB, not gray scale

<img src = "./images/09-08.png">

- Using the same filter

<img src = "./images/09-09.png">

<img src = "./images/09-10.png">

<img src = "./images/09-11.png">

<img src = "./images/09-12.png" width=10%>

<img src = "./images/09-08.png">

### 1.2 One Convolutioanl Layer: High Level View Summary

<img src = "./images/09-03.png">

- As the **kernel is sliding/convolving** across the image $\rightarrow$ 2 operations done **per patch**
    1. Element-wise multiplication
    2. Summation
- More **kernels** = more **feature map channels**
    - Can capture **more information** about the input

### 1.3 Multiple Convolutional Layers: High Level View

- deeper $\uparrow$ , the number of kernels $\uparrow$ => more informations

<img src = "./images/09-13.png">

### 1.4 Pooling Layer: High Level View

- 2 Common Types
    - Max Pooling
    - Average Pooling

<img src = "./images/09-14.png">

<img src = "./images/09-15.png">

<img src = "./images/09-16.png">

<img src = "./images/09-17.png">

<img src = "./images/09-18.png">

<img src = "./images/09-19.png">

<img src = "./images/09-20.png">

<img src = "./images/09-21.png">

<img src = "./images/09-22.png">

### 1.5 Multiple Pooling Layers: High Level View

<img src = "./images/09-13.png">

### 1.6 Padding